In [4]:
import pandas as pd
from inline_sql import sql, sql_val
from pandasql import sqldf

Sedes_Basico = pd.read_csv('Tablas/Sedes.csv')
Sedes = pd.read_csv('Tablas/Sedes.csv')
Secciones = pd.read_csv('Tablas/Secciones.csv')
Migraciones = pd.read_csv('Tablas/Migraciones.csv')

Sedes[['sede_id', 'redes_sociales']]

Redes_FB = [] # Facebook.
Redes_TW = [] # Twitter.
Redes_IG = [] # Instragram.
Redes_YT = [] # YouTube.
Redes_LI = [] # Linkedin.
Redes_FR = [] # Flickr.
Redes_Invalidas = [] # Links mal formados o @ ambiguos (Instagram o Twitter).
Redes_NULL = [] # Sede sin link.

Redes_Totales = []

for Sede_Id, Redes in zip(Sedes['sede_id'], Sedes['redes_sociales']):
    print(Redes)
    Redes = str(Redes).split('  //  ') # Lista con las redes sociales de esa sede (puede estar vacía).

    for Red in Redes:

        # Por el formato, si hay links el strip introduce un dato extra que siempre es el string vacío. Con esto, lo eliminamos.
        if(Red == ''): continue 

        Red = Red.strip() # Quitar espacios al inicio y al final.
        Red = Red[0].lower() + Red[1:] # Que empiece siempre en minus para mayor consistencia (hay datos que se escapaban a esto y se filtraban mal).

        if(Red == 'nan'):
            Redes_NULL.append(Sede_Id)
            continue
        else: Redes_Totales.append((Sede_Id, Red))

        if(Red[0] =='@' or ' ' in Red or not '.com' in Red): Redes_Invalidas.append((Sede_Id, Red))
        elif('facebook' in Red): Redes_FB.append((Sede_Id, Red))
        elif('twitter' in Red): Redes_TW.append((Sede_Id, Red))
        elif('instagram' in Red): Redes_IG.append((Sede_Id, Red))
        elif('linkedin' in Red): Redes_LI.append((Sede_Id, Red))
        elif('flickr' in Red): Redes_FR.append((Sede_Id, Red))
        elif('youtube' in Red): Redes_YT.append((Sede_Id, Red))

Redes_Validas = Redes_FB + Redes_TW + Redes_IG + Redes_LI + Redes_FR + Redes_YT

Redes_Descartadas = set(Redes_Totales).difference(set(Redes_FB)).difference(set(Redes_IG)).difference(set(Redes_LI)).difference(set(Redes_FR)).difference(set(Redes_YT)).difference(set(Redes_TW)).difference(set(Redes_Invalidas))

Sede_IDs = []
Redes_Sociales = []
URLs = []

Dic_Redes = {'sede_id': Sede_IDs, 'red_social': Redes_Sociales, 'url': URLs}

for Sede_Id, Url in Redes_FB:
    Dic_Redes['sede_id'].append(Sede_Id)
    Dic_Redes['red_social'].append('facebook')
    Dic_Redes['url'].append(Url)

for Sede_Id, Url in Redes_TW:
    Dic_Redes['sede_id'].append(Sede_Id)
    Dic_Redes['red_social'].append('twitter')
    Dic_Redes['url'].append(Url)

for Sede_Id, Url in Redes_IG:
    Dic_Redes['sede_id'].append(Sede_Id)
    Dic_Redes['red_social'].append('instagram')
    Dic_Redes['url'].append(Url)

for Sede_Id, Url in Redes_YT:
    Dic_Redes['sede_id'].append(Sede_Id)
    Dic_Redes['red_social'].append('youtube')
    Dic_Redes['url'].append(Url)

for Sede_Id, Url in Redes_FR:
    Dic_Redes['sede_id'].append(Sede_Id)
    Dic_Redes['red_social'].append('flickr')
    Dic_Redes['url'].append(Url)

for Sede_Id, Url in Redes_LI:
    Dic_Redes['sede_id'].append(Sede_Id)
    Dic_Redes['red_social'].append('linkedin')
    Dic_Redes['url'].append(Url)

Redes_DF = pd.DataFrame(Dic_Redes)

Sedes_Basico.sample(2)

# Consulta SQL.

Query =  '''
          SELECT s.pais_castellano AS país,
                 r.sede_id AS sede,
                 r."red_social",
                 r.url
          FROM Redes_DF AS r

          INNER JOIN Sedes_Basico AS s
          ON s.sede_id = r.sede_id

          ORDER BY país ASC, sede ASC, "red_social" ASC, url ASC
'''

Out = sqldf(Query)

Out.to_csv('redes_sociales_ejercicio_h_iv.csv')

https://twitter.com/ArgPortoAlegre  //  https://www.facebook.com/ArgentinaEnPortoAlegre/  //  https://www.instagram.com/argenportoalegre/  //  
https://www.facebook.com/Consulado-de-la-Rep%C3%BAblica-Argentina-en-Antofagasta-255728465291743/  //  
https://www.facebook.com/profile.php?id=100014152878593  //  
@ArgentinaEnAsuncion  //  
nan
facebook.com/ArgEnBarcelona  //  twitter.com/ARGenBarcelona  //  instagram.com/argenbarcelona  //  Consulado  Argentino  en  Barcelona  //  linkedin.com/company/consulado-y-centro-de-promoci%C3%B3n-de-la-republica-argentina-en-barcelona/mycompany/  //  
nan
https://www.facebook.com/ArgentinaEnBeloHorizonte/  //  https://twitter.com/consgralbelo  //  https://www.instagram.com/consgralbelo/  //  
https://www.facebook.com/ConsuladoArgentinoenBonn  //  http://www.instagram.com/consulado.argentino.bonn  //  
www.facebook.com/ArgentinaEnCadiz  //  
https://www.facebook.com/ArgEnCiudaddeleste/  //  https://www.instagram.com/argenciudaddeleste  //  
https://t